In [ ]:
!pip install ultralytics
!pip install torch torchvision


In [ ]:
!pip install opencv-python


In [24]:
import torch

# Check if CUDA (GPU) is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


Using device: cuda


In [25]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [26]:
import os
import cv2

# Define classes for detection
animals_detect = ["Cheetah", "Jaguar", "Leopard", "Lion", "Tiger"]
animals_encoding = {"Cheetah": 0, "Jaguar": 1, "Leopard": 2, "Lion": 3, "Tiger": 4}

# Define paths
train_dir = "/content/drive/MyDrive/wild_animal/archive/train"
test_dir = "/content/drive/MyDrive/wild_animal/archive/test"

# Create YOLO directory structure
os.makedirs("yolo/train/images", exist_ok=True)
os.makedirs("yolo/train/labels", exist_ok=True)
os.makedirs("yolo/test/images", exist_ok=True)
os.makedirs("yolo/test/labels", exist_ok=True)

# Image resize dimensions
size = (640, 640)

# Process train images
for animal_specie in animals_detect:
    image_files = os.listdir(os.path.join(train_dir, animal_specie))
    for img_file in image_files:
        if img_file != "Label" and img_file.endswith(".jpg"):  # Ignore the Label folder
            img_path = os.path.join(train_dir, animal_specie, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Skipping corrupted image: {img_file}")
                continue
            img = cv2.resize(img, size)
            cv2.imwrite(f"yolo/train/images/{img_file}", img)

# Process test images
for animal_specie in animals_detect:
    image_files = os.listdir(os.path.join(test_dir, animal_specie))
    for img_file in image_files:
        if img_file != "Label" and img_file.endswith(".jpg"):
            img_path = os.path.join(test_dir, animal_specie, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Skipping corrupted image: {img_file}")
                continue
            img = cv2.resize(img, size)
            cv2.imwrite(f"yolo/test/images/{img_file}", img)

print("✅ Image processing complete.")


✅ Image processing complete.


In [27]:
def process_files(input_files_path, output_files_path):
    for animal_specie in animals_detect:
        label_dir = os.path.join(input_files_path, animal_specie, "Label")
        if not os.path.exists(label_dir):
            print(f"Skipping {animal_specie}, no labels found.")
            continue

        txt_files = os.listdir(label_dir)
        for txt_file in txt_files:
            if not txt_file.endswith(".txt"):
                continue

            img_filename = txt_file.replace(".txt", ".jpg")
            img_path = os.path.join(input_files_path, animal_specie, img_filename)

            if not os.path.exists(img_path):
                print(f"Skipping {txt_file}, matching image not found.")
                continue

            img = cv2.imread(img_path)
            if img is None:
                print(f"Skipping corrupted image: {img_filename}")
                continue

            height, width, _ = img.shape
            with open(os.path.join(label_dir, txt_file), "r") as source:
                with open(os.path.join(output_files_path, txt_file), "w") as destination:
                    for line in source:
                        parts = line.strip().split()
                        if len(parts) < 5:
                            print(f"Skipping malformed label file: {txt_file}")
                            continue

                        class_name = parts[0]
                        if class_name not in animals_encoding:
                            print(f"Skipping unknown class {class_name} in {txt_file}")
                            continue

                        class_id = animals_encoding[class_name]
                        xmin, ymin, xmax, ymax = map(float, parts[1:])

                        # Convert to YOLO format (normalized values)
                        x_center = (xmin + xmax) / (2 * width)
                        y_center = (ymin + ymax) / (2 * height)
                        box_width = (xmax - xmin) / width
                        box_height = (ymax - ymin) / height

                        destination.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")

# Convert train labels
process_files(train_dir, "yolo/train/labels")

# Convert test labels
process_files(test_dir, "yolo/test/labels")

print("✅ Label conversion complete.")


✅ Label conversion complete.


In [28]:
data_yaml = """train: /content/yolo/train/images
val: /content/yolo/test/images

nc: 5
names: ["Cheetah", "Jaguar", "Leopard", "Lion", "Tiger"]
"""

with open("yolo/data.yaml", "w") as f:
    f.write(data_yaml)

print("✅ data.yaml created.")

✅ data.yaml created.


In [32]:
from ultralytics import YOLO

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = YOLO("yolo11n.pt")  # Use YOLO11 if available

model.train(
    data="yolo/data.yaml",
    epochs=50,
    batch=64,
    imgsz=640,
    device=device,
    lr0=0.00001,
)


Using device: cuda
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=yolo/data.yaml, epochs=50, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

train: Scanning /content/yolo/train/labels.cache... 764 images, 0 backgrounds, 0 corrupt: 100%|██████████| 764/764 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/yolo/test/labels.cache... 190 images, 0 backgrounds, 0 corrupt: 100%|██████████| 190/190 [00:00<?, ?it/s]


Plotting labels to runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=1e-05' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train6
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        190        199      0.536      0.768      0.619      0.528



50 epochs completed in 0.246 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 5.5MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.34s/it]


                   all        190        199      0.661      0.758      0.641      0.548
               Cheetah          5          5      0.171          1      0.248      0.194
                Jaguar          5          5      0.319        0.6      0.211      0.182
               Leopard         55         59      0.949      0.441      0.885      0.725
                  Lion         99        103      0.951      0.951      0.975      0.852
                 Tiger         26         27      0.915        0.8      0.884      0.789
Speed: 0.3ms preprocess, 2.4ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a027026dd10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [34]:
import shutil

# Define the path where YOLO saves the model
saved_model_path = "runs/detect/train4/weights/best.pt"  # Change 'train4' if needed
destination_path = "/content/drive/MyDrive/yolo11_trained_model.pt"  # Save in Google Drive

# Copy the model to Google Drive
shutil.copy(saved_model_path, destination_path)

print(f"✅ Model saved to {destination_path}")

✅ Model saved to /content/drive/MyDrive/yolo11_trained_model.pt


In [33]:
metrics = model.val(data="yolo/data.yaml")
print(metrics)


Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/yolo/test/labels.cache... 190 images, 0 backgrounds, 0 corrupt: 100%|██████████| 190/190 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:07<00:00,  2.43s/it]


                   all        190        199      0.661      0.758      0.641      0.549
               Cheetah          5          5      0.171          1      0.248      0.194
                Jaguar          5          5       0.32        0.6      0.211      0.182
               Leopard         55         59      0.948      0.441      0.885      0.725
                  Lion         99        103      0.951      0.951      0.975       0.85
                 Tiger         26         27      0.915      0.799      0.884      0.793
Speed: 8.3ms preprocess, 4.9ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train62
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a026ed7dd10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_resu